# Data Wrangling

This notebook presents different data wrangling techniques used commonly

In [1]:
# import required libraries
import random
import datetime 
import numpy as np
import pandas as pd
from random import randrange
from sklearn import preprocessing

from IPython.display import display

pd.options.mode.chained_assignment = None

## Utilities

In [2]:
def _random_date(start,date_count):
    """This function generates a random date based on params
    Args:
        start (date object): the base date
        date_count (int): number of dates to be generated
    Returns:
        list of random dates

    """
    current = start
    while date_count > 0:
        curr = current + datetime.timedelta(days=randrange(42))
        yield curr
        date_count-=1


def generate_sample_data(row_count=100):
    """This function generates a random transaction dataset
    Args:
        row_count (int): number of rows for the dataframe
    Returns:
        a pandas dataframe

    """
    
    # sentinels
    startDate = datetime.datetime(2016, 1, 1,13)
    serial_number_sentinel = 1000
    user_id_sentinel = 5001
    product_id_sentinel = 101
    price_sentinel = 2000
    
    
    # base list of attributes
    data_dict = {
    'Serial No': np.arange(row_count)+serial_number_sentinel,
    'Date': np.random.permutation(pd.to_datetime([x.strftime("%d-%m-%Y") 
                                                    for x in _random_date(startDate,
                                                                          row_count)]).date
                                  ),
    'User ID': np.random.permutation(np.random.randint(0,
                                                       row_count,
                                                       size=int(row_count/10)) + user_id_sentinel).tolist()*10,
    'Product ID': np.random.permutation(np.random.randint(0,
                                                          row_count,
                                                          size=int(row_count/10))+ product_id_sentinel).tolist()*10 ,
    'Quantity Purchased': np.random.permutation(np.random.randint(1,
                                                                  42,
                                                                  size=row_count)),
    'Price': np.round(np.abs(np.random.randn(row_count)+1)*price_sentinel,
                      decimals=2),
    'User Type':np.random.permutation([chr(random.randrange(97, 97 + 3 + 1)) 
                                            for i in range(row_count)])
    }
    
    # introduce missing values
    for index in range(int(np.sqrt(row_count))): 
        data_dict['Price'][np.argmax(data_dict['Price'] == random.choice(data_dict['Price']))] = np.nan
        data_dict['User Type'][np.argmax(data_dict['User Type'] == random.choice(data_dict['User Type']))] = np.nan
        data_dict['Date'][np.argmax(data_dict['Date'] == random.choice(data_dict['Date']))] = np.nan
        data_dict['Product ID'][np.argmax(data_dict['Product ID'] == random.choice(data_dict['Product ID']))] = 0
        data_dict['Serial No'][np.argmax(data_dict['Serial No'] == random.choice(data_dict['Serial No']))] = -1
        data_dict['User ID'][np.argmax(data_dict['User ID'] == random.choice(data_dict['User ID']))] = -101
        
    
    # create data frame
    df = pd.DataFrame(data_dict)
    
    return df
    

def describe_dataframe(df=pd.DataFrame()):
    """This function generates descriptive stats of a dataframe
    Args:
        df (dataframe): the dataframe to be analyzed
    Returns:
        None

    """
    print("\n\n")
    print("*"*30)
    print("About the Data")
    print("*"*30)
    
    print("Number of rows::",df.shape[0])
    print("Number of columns::",df.shape[1])
    print("\n")
    
    print("Column Names::",df.columns.values.tolist())
    print("\n")
    
    print("Column Data Types::\n",df.dtypes)
    print("\n")
    
    print("Columns with Missing Values::",df.columns[df.isnull().any()].tolist())
    print("\n")
    
    print("Number of rows with Missing Values::",len(pd.isnull(df).any(1).nonzero()[0].tolist()))
    print("\n")
    
    print("Sample Indices with missing data::",pd.isnull(df).any(1).nonzero()[0].tolist()[0:5])
    print("\n")
    
    print("General Stats::")
    print(df.info())
    print("\n")
    
    print("Summary Stats::")
    print(df.describe())
    print("\n")
    
    print("Dataframe Sample Rows::")
    display(df.head(5))
    
def cleanup_column_names(df,rename_dict={},do_inplace=True):
    """This function renames columns of a pandas dataframe
       It converts column names to snake case if rename_dict is not passed. 
    Args:
        rename_dict (dict): keys represent old column names and values point to 
                            newer ones
        do_inplace (bool): flag to update existing dataframe or return a new one
    Returns:
        pandas dataframe if do_inplace is set to False, None otherwise

    """
    if not rename_dict:
        return df.rename(columns={col: col.lower().replace(' ','_') 
                    for col in df.columns.values.tolist()}, 
                  inplace=do_inplace)
    else:
        return df.rename(columns=rename_dict,inplace=do_inplace)

def expand_user_type(u_type):
    """This function maps user types to user classes
    Args:
        u_type (str): user type value
    Returns:
        (str) user_class value

    """
    if u_type in ['a','b']:
        return 'new'
    elif u_type == 'c':
        return 'existing'
    elif u_type == 'd':
        return 'loyal_existing'
    else:
        return 'error'

## Generate a Sample Dataset

In [3]:
df = generate_sample_data(row_count=1000)

### Describe the Dataset

In [4]:
describe_dataframe(df)




******************************
About the Data
******************************
Number of rows:: 1000
Number of columns:: 7


Column Names:: ['Serial No', 'Date', 'User ID', 'Product ID', 'Quantity Purchased', 'Price', 'User Type']


Column Data Types::
 Serial No               int32
Date                   object
User ID                 int64
Product ID              int64
Quantity Purchased      int32
Price                 float64
User Type              object
dtype: object


Columns with Missing Values:: ['Date', 'Price']


Number of rows with Missing Values:: 60


Sample Indices with missing data:: [0, 1, 2, 4, 6]


General Stats::
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
Serial No             1000 non-null int32
Date                  969 non-null object
User ID               1000 non-null int64
Product ID            1000 non-null int64
Quantity Purchased    1000 non-null int32
Price                 969 non-null float64


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:98: FutureWarning: Series.nonzero() is deprecated and will be removed in a future version.Use Series.to_numpy().nonzero() instead
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:101: FutureWarning: Series.nonzero() is deprecated and will be removed in a future version.Use Series.to_numpy().nonzero() instead


         Serial No      User ID  Product ID  Quantity Purchased        Price
count  1000.000000  1000.000000  1000.00000         1000.000000   969.000000
mean   1453.048000  5474.245000   608.29900           21.027000  2316.372900
std     385.573511   329.968414   273.15162           12.065233  1601.936658
min      -1.000000  -101.000000     0.00000            1.000000     9.770000
25%    1225.750000  5216.000000   368.00000           11.000000   989.320000
50%    1484.500000  5471.000000   600.00000           21.000000  2054.920000
75%    1741.250000  5702.000000   817.00000           32.000000  3340.630000
max    1999.000000  5998.000000  1095.00000           41.000000  7796.160000


Dataframe Sample Rows::


,Serial No,Date,User ID,Product ID,Quantity Purchased,Price,User Type
0,1000,NaN,-101,0,15,3638.78,n
1,1001,NaN,5140,227,24,1091.51,n
2,1002,NaN,5156,719,32,2821.70,n
3,1003,2016-01-29,5544,302,17,449.21,n
4,1004,NaN,5454,638,16,NaN,n


### Rename Columns

In [5]:
print("Dataframe columns:\n{}".format(df.columns.tolist()))

Dataframe columns:
['Serial No', 'Date', 'User ID', 'Product ID', 'Quantity Purchased', 'Price', 'User Type']


In [6]:
cleanup_column_names(df)

In [7]:
print("Dataframe columns:\n{}".format(df.columns.tolist()))

Dataframe columns:
['serial_no', 'date', 'user_id', 'product_id', 'quantity_purchased', 'price', 'user_type']


### Sort Rows on defined attributes

In [8]:
display(df.sort_values(['serial_no', 'price'], 
                         ascending=[True, False]).head())

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type
137,-1,2016-07-02,5324,689,39,3392.47,d
878,-1,2016-03-01,5182,430,26,3210.33,d
126,-1,2016-07-02,5267,312,31,3116.61,d
20,-1,2016-05-01,5317,266,34,2776.11,d
111,-1,2016-05-01,5079,324,31,2714.92,d


### Rearrange Columns in a Dataframe

In [9]:
display(df[['serial_no','date','user_id','user_type',
              'product_id','quantity_purchased','price']].head())

,serial_no,date,user_id,user_type,product_id,quantity_purchased,price
0,1000,NaN,-101,n,0,15,3638.78
1,1001,NaN,5140,n,227,24,1091.51
2,1002,NaN,5156,n,719,32,2821.70
3,1003,2016-01-29,5544,n,302,17,449.21
4,1004,NaN,5454,n,638,16,NaN


### Filtering Columns

Using Column Index

In [10]:
# print 10 values from column at index 3
print(df.iloc[:,3].values[0:10])

[  0 227 719 302 638 273 182 264 714 735]


Using Column Name

In [11]:
# print 10 values of quantity purchased
print(df.quantity_purchased.values[0:10])

[15 24 32 17 16 29  1 11 33  8]


Using Column Datatype

In [12]:
# print 10 values of columns with data type float
print(df.select_dtypes(include=['float64']).values[:10,0])

[3638.78 1091.51 2821.7   449.21     nan 4804.55  466.03   76.3  4044.19
 2324.46]


### Filtering Rows

Select specific rows

In [13]:
display(df.iloc[[10,501,20]])

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type
10,1010,NaN,5448,878,5,4756.52,n
501,1501,2016-09-02,5140,227,20,167.45,b
20,-1,2016-05-01,5317,266,34,2776.11,d


Exclude Specific Row indices

In [14]:
display(df.drop([0,24,51], axis=0).head())

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type
1,1001,NaN,5140,227,24,1091.51,n
2,1002,NaN,5156,719,32,2821.70,n
3,1003,2016-01-29,5544,302,17,449.21,n
4,1004,NaN,5454,638,16,NaN,n
5,1005,2016-10-01,5482,273,29,4804.55,n


Conditional Filtering

In [15]:
display(df[df.quantity_purchased>25].head())

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type
2,1002,NaN,5156,719,32,2821.70,n
5,1005,2016-10-01,5482,273,29,4804.55,n
8,1008,2016-01-13,5049,714,33,4044.19,n
11,1011,2016-01-01,5079,324,33,3324.25,n
12,1012,2016-01-20,5238,878,36,249.26,n


Offset from top of the dataframe

In [16]:
display(df[100:].head())

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type
100,1100,2016-10-01,5264,891,39,867.78,a
101,1101,2016-01-02,5140,227,21,4323.52,a
102,1102,2016-12-01,5156,719,25,2731.98,d
103,1103,2016-07-01,5544,302,6,1463.81,d
104,1104,2016-09-02,5454,638,1,7312.82,c


Offset from bottom of the dataframe

In [17]:
display(df[-10:].head())

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type
990,1990,2016-11-02,5810,764,33,909.48,c
991,1991,2016-08-02,5362,655,36,1005.21,c
992,-1,2016-01-28,5021,569,39,1808.09,a
993,1993,2016-11-01,5951,817,16,289.65,c
994,1994,2016-01-29,5092,336,27,3161.08,a


### TypeCasting/Data Type Conversion

In [18]:
df['date'] = pd.to_datetime(df.date)
# compare dtypes of the original df with this one
print(df.dtypes)

serial_no                      int32
date                  datetime64[ns]
user_id                        int64
product_id                     int64
quantity_purchased             int32
price                        float64
user_type                     object
dtype: object


### Apply/Map Usage

Map : Create a derived attribute using map

In [19]:
df['user_class'] = df['user_type'].map(expand_user_type)
display(df.tail())

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type,user_class
995,1995,2016-10-01,5456,648,3,4636.60,b,new
996,1996,2016-01-25,5725,443,13,3283.50,d,loyal_existing
997,1997,2016-10-02,5998,579,38,1326.40,a,new
998,1998,2016-01-24,5442,459,35,635.31,a,new
999,1999,2016-08-02,5871,742,9,6385.19,a,new


Apply: Using apply to get attribute ranges

In [20]:
display(df.select_dtypes(include=[np.number]).apply(lambda x: 
                                                        x.max()- x.min()))

serial_no             2000.00
user_id               6099.00
product_id            1095.00
quantity_purchased      40.00
price                 7786.39
dtype: float64

Applymap: Extract week from date

In [21]:
df['purchase_week'] = df[['date']].applymap(lambda dt:dt.week 
                                                if not pd.isnull(dt.week) 
                                                else 0)

In [22]:
display(df.head())

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type,user_class,purchase_week
0,1000,NaT,-101,0,15,3638.78,n,error,0
1,1001,NaT,5140,227,24,1091.51,n,error,0
2,1002,NaT,5156,719,32,2821.70,n,error,0
3,1003,2016-01-29,5544,302,17,449.21,n,error,4
4,1004,NaT,5454,638,16,NaN,n,error,0


### Missing Values

Drop Rows with missing dates

In [23]:
df_dropped = df.dropna(subset=['date'])
display(df_dropped.head())

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type,user_class,purchase_week
3,1003,2016-01-29,5544,302,17,449.21,n,error,4
5,1005,2016-10-01,5482,273,29,4804.55,n,error,39
8,1008,2016-01-13,5049,714,33,4044.19,n,error,2
11,1011,2016-01-01,5079,324,33,3324.25,n,error,53
12,1012,2016-01-20,5238,878,36,249.26,n,error,3


Fill Missing Price values with mean price

In [24]:
df_dropped['price'].fillna(value=np.round(df.price.mean(),decimals=2),
                                inplace=True)

Fill Missing user_type values with value from previous row (forward fill) 

In [25]:
df_dropped['user_type'].fillna(method='ffill',inplace=True)

Fill Missing user_type values with value from next row (backward fill)

In [26]:
df_dropped['user_type'].fillna(method='bfill',inplace=True)

### Duplicates

Drop Duplicate serial_no rows

In [27]:
# sample duplicates
display(df_dropped[df_dropped.duplicated(subset=['serial_no'])].head())
print("Shape of df={}".format(df_dropped.shape))

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type,user_class,purchase_week
87,-1,2016-01-14,5501,294,1,537.79,a,new,2
111,-1,2016-05-01,5079,324,31,2714.92,d,loyal_existing,17
126,-1,2016-07-02,5267,312,31,3116.61,d,loyal_existing,26
133,-1,2016-01-20,5921,473,21,515.52,d,loyal_existing,3
137,-1,2016-07-02,5324,689,39,3392.47,d,loyal_existing,26


Shape of df=(969, 9)


In [28]:
df_dropped.drop_duplicates(subset=['serial_no'],inplace=True)

In [29]:
# updated dataframe
display(df_dropped.head())
print("Shape of df={}".format(df_dropped.shape))

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type,user_class,purchase_week
3,1003,2016-01-29,5544,302,17,449.21,n,error,4
5,1005,2016-10-01,5482,273,29,4804.55,n,error,39
8,1008,2016-01-13,5049,714,33,4044.19,n,error,2
11,1011,2016-01-01,5079,324,33,3324.25,n,error,53
12,1012,2016-01-20,5238,878,36,249.26,n,error,3


Shape of df=(939, 9)


Remove rows which have less than 3 attributes with non-missing data

In [30]:
display(df.dropna(thresh=3).head())
print("Shape of df={}".format(df.dropna(thresh=3).shape))

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type,user_class,purchase_week
0,1000,NaT,-101,0,15,3638.78,n,error,0
1,1001,NaT,5140,227,24,1091.51,n,error,0
2,1002,NaT,5156,719,32,2821.70,n,error,0
3,1003,2016-01-29,5544,302,17,449.21,n,error,4
4,1004,NaT,5454,638,16,NaN,n,error,0


Shape of df=(1000, 9)


### Encode Categoricals

One Hot Encoding using get_dummies()

In [31]:
display(pd.get_dummies(df,columns=['user_type']).head())

,serial_no,date,user_id,product_id,quantity_purchased,price,user_class,purchase_week,user_type_a,user_type_b,user_type_c,user_type_d,user_type_n
0,1000,NaT,-101,0,15,3638.78,error,0,0,0,0,0,1
1,1001,NaT,5140,227,24,1091.51,error,0,0,0,0,0,1
2,1002,NaT,5156,719,32,2821.70,error,0,0,0,0,0,1
3,1003,2016-01-29,5544,302,17,449.21,error,4,0,0,0,0,1
4,1004,NaT,5454,638,16,NaN,error,0,0,0,0,0,1


Label Mapping

In [32]:
type_map={'a':0,'b':1,'c':2,'d':3,np.NAN:-1}
df['encoded_user_type'] = df.user_type.map(type_map)
display((df.tail()))

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type,user_class,purchase_week,encoded_user_type
995,1995,2016-10-01,5456,648,3,4636.60,b,new,39,1.0
996,1996,2016-01-25,5725,443,13,3283.50,d,loyal_existing,4,3.0
997,1997,2016-10-02,5998,579,38,1326.40,a,new,39,0.0
998,1998,2016-01-24,5442,459,35,635.31,a,new,3,0.0
999,1999,2016-08-02,5871,742,9,6385.19,a,new,31,0.0


### Random Sampling data from DataFrame

In [33]:
display(df.sample(frac=0.2, replace=True, random_state=42).head())

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type,user_class,purchase_week,encoded_user_type
102,1102,2016-12-01,5156,719,25,2731.98,d,loyal_existing,48,3.0
435,-1,2016-12-01,5473,1055,33,1643.29,c,existing,48,2.0
860,1860,2016-08-01,5372,813,41,1670.04,a,new,31,0.0
270,1270,2016-01-20,5638,391,2,4677.35,c,existing,3,2.0
106,1106,2016-06-02,5699,182,38,1421.38,a,new,22,0.0


### Normalizing Numeric Values

Normalize price values using  **Min-Max Scaler**

In [34]:
df_normalized = df.dropna().copy()
min_max_scaler = preprocessing.MinMaxScaler()
np_scaled = min_max_scaler.fit_transform(df_normalized['price'].values.reshape(-1,1))
df_normalized['price'] = np_scaled.reshape(-1,1)

In [35]:
display(df_normalized.head())

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type,user_class,purchase_week,encoded_user_type
18,1018,2016-01-16,5722,166,22,0.658426,d,loyal_existing,2,3.0
20,-1,2016-05-01,5317,266,34,0.355279,d,loyal_existing,17,3.0
26,1026,2016-12-01,5267,312,29,0.139508,b,new,48,1.0
28,1028,2016-07-01,5144,1035,35,0.006800,a,new,26,0.0
30,1030,2016-01-28,5718,135,6,0.090963,a,new,4,0.0


Normalize quantity purchased values using  **Robust Scaler**

In [36]:
df_normalized = df.dropna().copy()
robust_scaler = preprocessing.RobustScaler()
rs_scaled = robust_scaler.fit_transform(df_normalized['quantity_purchased'].values.reshape(-1,1))
df_normalized['quantity_purchased'] = rs_scaled.reshape(-1,1)

In [37]:
display(df_normalized.head())

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type,user_class,purchase_week,encoded_user_type
18,1018,2016-01-16,5722,166,0.05,5136.53,d,loyal_existing,2,3.0
20,-1,2016-05-01,5317,266,0.65,2776.11,d,loyal_existing,17,3.0
26,1026,2016-12-01,5267,312,0.40,1096.03,b,new,48,1.0
28,1028,2016-07-01,5144,1035,0.70,62.72,a,new,26,0.0
30,1030,2016-01-28,5718,135,-0.75,718.04,a,new,4,0.0


### Data Summarization

Condition based aggregation

In [38]:
print("Mean price of items purchased by user_type=a :: {}".format(df['price'][df['user_type']=='a'].mean()))

Mean price of items purchased by user_type=a :: 2427.206591928251


Condtion based counts

In [39]:
print(df['purchase_week'].value_counts())

4     169
3     164
2     109
53     61
26     59
44     53
39     50
31     46
9      43
5      43
35     38
22     36
13     34
0      31
17     25
48     21
18     18
Name: purchase_week, dtype: int64


### Group By

Group By certain attributes

In [40]:
print(df.groupby(['user_class'])['quantity_purchased'].sum())

user_class
error              641
existing          5296
loyal_existing    5120
new               9970
Name: quantity_purchased, dtype: int32


Group By with different aggregate functions

In [41]:
display(df.groupby(['user_class'])['quantity_purchased'].agg([np.sum,
                                                                np.mean,
                                                                np.count_nonzero]))

,sum,mean,count_nonzero
user_class,,,
error,641,20.677419,31
existing,5296,21.704918,244
loyal_existing,5120,21.069959,243
new,9970,20.684647,482


Group by specific aggregate functions for each attribute

In [42]:
display(df.groupby(['user_class','user_type']).agg({'price':np.mean,
                                                        'quantity_purchased':np.max}))

price  quantity_purchased
user_class     user_type                                 
error          n          2188.463000                  41
existing       c          2217.720802                  41
loyal_existing d          2431.726176                  41
new            a          2427.206592                  41
               b          2212.837012                  41

Group by with multiple agg for each attribute

In [43]:
display(df.groupby(['user_class','user_type']).agg({'price':{
                                                                'total_price':np.sum,
                                                                'mean_price':np.mean,
                                                                'variance_price':np.std,
                                                                'count':np.count_nonzero},
                                                   'quantity_purchased':np.sum}))  

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\groupby\generic.py:1315: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


price                                     \
                         total_price   mean_price variance_price  count   
user_class     user_type                                                  
error          n            65653.89  2188.463000    1408.883457   31.0   
existing       c           525599.83  2217.720802    1574.674356  244.0   
loyal_existing d           578750.83  2431.726176    1695.385543  243.0   
new            a           541267.07  2427.206592    1646.084423  234.0   
               b           533293.72  2212.837012    1510.104718  248.0   

                         quantity_purchased  
                                        sum  
user_class     user_type                     
error          n                        641  
existing       c                       5296  
loyal_existing d                       5120  
new            a                       4926  
               b                       5044

### Pivot Tables

In [44]:
display(df.pivot_table(index='date', columns='user_type', 
                         values='price',aggfunc=np.mean))

user_type,a,b,c,d,n
date,,,,,
2016-01-01,2380.440000,1643.915000,2355.837500,1447.802000,3234.225
2016-01-02,2981.280000,1916.397500,3365.202500,2701.442000,2826.570
2016-01-13,2989.870000,2297.532500,481.146667,2318.262500,4044.190
2016-01-14,2199.440000,1727.352500,2511.209000,2588.711429,NaN
2016-01-15,2058.838000,1382.173333,2645.193333,1809.301429,NaN
2016-01-16,3448.000000,1350.590000,2405.938000,3531.723333,NaN
2016-01-17,2287.816250,1906.420000,1862.272000,2690.474000,NaN
2016-01-18,3795.742857,2051.252500,3120.033333,2400.090000,589.790
2016-01-19,2451.903333,1368.910000,1705.667143,191.185000,NaN


### Stack a Dataframe

In [45]:
print(df.stack())

0    serial_no                            1000
     user_id                              -101
     product_id                              0
     quantity_purchased                     15
     price                             3638.78
     user_type                               n
     user_class                          error
     purchase_week                           0
1    serial_no                            1001
     user_id                              5140
     product_id                            227
     quantity_purchased                     24
     price                             1091.51
     user_type                               n
     user_class                          error
     purchase_week                           0
2    serial_no                            1002
     user_id                              5156
     product_id                            719
     quantity_purchased                     32
     price                              2821.7
     user_typ